In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!pip install transformers

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import shutil
import sys
import zipfile
from sklearn.model_selection import train_test_split

In [69]:
file_path = "/content/drive/MyDrive/my-datasets/yektanet_train.csv"

data = pd.read_csv(file_path)

features = data.drop('category', axis=1)
target = data['category']

x_train, x_test, y_train, y_test = train_test_split(
    features, target, test_size=0.2, random_state=42, stratify=target
)

print("Training set shape:", x_train.shape)
print("Test set shape:", x_test.shape)
x_train.head()

Training set shape: (3831, 8)
Test set shape: (958, 8)


,description,text_content,title,h1,h2,url,domain,id
1334,وحید شمسایی، سرمربی تیم ملی فوتسال که به خاطر ...,شمسایی به تیم ملی بازگشت سرمربی تیم ملی فوتسال...,شمسایی به تیم ملی بازگشت,شمسایی به تیم ملی بازگشت,NaN,asriran.com/fa/news/856021/%D8%B4%D9%85%D8%B3%...,asriran.com,1759
2618,اقتصادنیوز: نماینده اصلاح طلب مجلس با تاکید بر...,پزشکیان: درباره پرونده «مهسا امینی» با مردم شف...,پزشکیان: درباره پرونده «مهسا امینی» با مردم شف...,پزشکیان: درباره پرونده «مهسا امینی» با مردم شف...,NaN,eghtesadnews.com/%D8%A8%D8%AE%D8%B4-%D8%A7%D8%...,eghtesadnews.com,1387
2812,معاون اول قوه قضائیه گفت: از همکاران قضایی است...,دستگاه قضایی استان فارس با سرعت و قاطعیت با عا...,دستگاه قضایی استان فارس با سرعت و قاطعیت با عا...,دستگاه قضایی استان فارس با سرعت و قاطعیت با عا...,وبگردی عناوین مرتبط نظر شما پربیننده‌ترین اخبا...,borna.news/%D8%A8%D8%AE%D8%B4-%D8%B3%DB%8C%D8%...,borna.news,1344
2849,پاپس کیک یکی از انواع کیک های خوشمزه و بین الم...,طرز تهیه پاپس کیک ساده و خوشمزه به روش خانگی چ...,طرز تهیه پاپس کیک ساده و خوشمزه به روش خانگی,طرز تهیه پاپس کیک ساده و خوشمزه به روش خانگی,طرز تهیه پاپس کیک,zendegi.online/41122/,zendegi.online,337
1085,رست بیف چیست رُست بیف یا Roast beef به معنای گ...,"رست بیف چیست - هلث کده رست بیف چیستسپتامبر 14,...",رست بیف چیست - هلث کده,هلث کده هلث کده رست بیف چیست رست بیف چیست,رست بیف چیست رست بيف چيست پیتزا رست بیف چیست س...,healthkade.info/%D8%B1%D8%B3%D8%AA-%D8%A8%DB%8...,healthkade.info,693


In [70]:
# drop useless columns
cols_to_drop = ["url", "domain", "id"]
existing_cols = [col for col in cols_to_drop if col in x_train.columns]
x_train.drop(labels=existing_cols, axis=1, inplace=True)

In [71]:
# implement hypo parameters
MAX_LEN = 256
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 2
LEARNING_RATE = 1e-05

In [72]:
from transformers import BertTokenizer,BertModel

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [73]:
# [CLS] , [PAD] , [SEP] , [MASK]

example_text = "My name is hossein"

encodings = tokenizer.encode_plus(
    example_text,
    add_special_tokens=True,
    max_length=MAX_LEN,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
)

encodings

{'input_ids': tensor([[  101,  2026,  2171,  2003,  7570, 11393,  2378,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [85]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.title = df['description']
        self.targets = self.df['category'].values

    def __len__(self):
        return len(self.title)

    def __getitem__(self,index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(), # [1, 512] => [512]
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs['token_type_ids'].flatten(),
            'targets': torch.FloatTensor(self.targets[index]),
        }

In [86]:
train_size = 0.8

train_df = pd.concat([x_train, y_train], axis=1)
train_df = train_df.sample(frac=train_size,random_state=200).reset_index(drop=True)

val_df = pd.concat([x_test, y_test], axis=1)
val_df = val_df.drop(x_test.index).reset_index(drop=True)

In [87]:
train_dataset = CustomDataset(train_df,tokenizer,MAX_LEN)
valid_dataset = CustomDataset(val_df,tokenizer,MAX_LEN)

In [88]:
train_data_loader = torch.utils.data.DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=TRAIN_BATCH_SIZE,
    num_workers=0
)

val_data_loader = torch.utils.data.DataLoader(
    valid_dataset,
    shuffle=False,
    batch_size=VALID_BATCH_SIZE,
    num_workers=0
)

In [89]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cpu


In [90]:
def load_ckp(checkpoint_fpath,model,optimizer):
    checkpoint = torch.load(checkpoint_fpath)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    valid_loss_min = checkpoint['valid_loss_min']
    return model, optimizer, checkpoint['epoch'],valid_loss_min.item()


def save_ckp(state,is_best,checkpoint_path,best_model_path):
    f_path = checkpoint_path
    torch.save(state,f_path)
    if is_best:
        best_fpath = best_model_path
        shutil.copyfile(f_path,best_fpath)

In [91]:
class BERTClass(nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased',return_dict=True)
        self.dropout = nn.Dropout(0.3)
        self.linear = nn.Linear(768,6)

    def forward(self,input_ids,attention_mask,token_type_ids):
      output = self.bert_model(input_ids,attention_mask,token_type_ids)
      output_droput = self.dropout(output.pooler_output)
      output = self.linear(output_dropout)
      return output

model = BERTClass()
model.to(device)

BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi